# Duty Scheduler

## Input Data

In [1]:
from datetime import date as d
from datetime import datetime as dt
import random
import sys
!{sys.executable} -m pip install icalendar
from icalendar import Calendar, Event, vCalAddress, vText
import uuid

In [2]:
random.seed(str(dt.now()))

In [3]:
sched_cal = Calendar()
sched_cal['summary'] = "Duty Schedule"
sched_cal.add('prodid', '-//My calendar product//mxm.dk//')

sched_cal.add('version', '2.0')

In [4]:
people = [
    {"name": "Ryan",
     "email": "rgilm@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-02'),
        d.fromisoformat('2023-11-05'),
        d.fromisoformat('2023-11-22'),
        d.fromisoformat('2023-11-15'),
        d.fromisoformat('2023-11-08'),
    ]},
    {"name": "Annabel",
     "email": "annabel@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-05'),
        d.fromisoformat('2023-11-10'),
        d.fromisoformat('2023-11-19'),
        d.fromisoformat('2023-11-11'),
    ]},
    {"name": "Bridget",
     "email": "bridget@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-01'),
        d.fromisoformat('2023-11-16'),
        d.fromisoformat('2023-11-27'),
        d.fromisoformat('2023-11-28'),
    ]},
    {"name": "Paul",
     "email": "paul@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-27'),
        d.fromisoformat('2023-11-28'),
    ]},
    {"name": "Chloe",
     "email": "chloe@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-09'),
        d.fromisoformat('2023-11-10'),
        d.fromisoformat('2023-11-21'),
        d.fromisoformat('2023-11-22'),
        d.fromisoformat('2023-11-23'),
        d.fromisoformat('2023-11-24'),
        d.fromisoformat('2023-11-25'),
    ]},
]

In [5]:
month_to_schedule = d(2023, 11, 1)

### create schedule dates 
use the month of the year to create a list of objects that contain the date, empty primary and secondary fields, whether it is a weekday or not, and a list of people who can work those days.

In [6]:
schedule = []
for date_of_month in range(1,31):
    t_date = d(month_to_schedule.year, month_to_schedule.month, date_of_month)
    avail_people=[]
    for person in people:
        if t_date not in person['unavail']:
            avail_people.append(person)
    schedule.append({"date": t_date, "avail_people": avail_people,"prim": None, "sec": None, "is_weekday": True if d.weekday(t_date) < 3 or d.weekday(t_date) > 5 else False})

In [7]:
# function for sorting keys

def get_duty_count_by_p_wd(person):
    return person['p_wd']

def get_duty_count_by_p_we(person):
    return person['p_we']

def get_duty_count_by_s_wd(person):
    return person['s_wd']

def get_duty_count_by_s_we(person):
    return person['s_we']

In [8]:
# sort people in day by selected shift and daytype

def sort_people_in_day(date, prim):
    random.shuffle(date['avail_people'])
    if prim is True and date['is_weekday'] is True:
        return date['avail_people'].sort(key=get_duty_count_by_p_wd)
    elif prim is True and date['is_weekday'] is False:
        return date['avail_people'].sort(key=get_duty_count_by_p_we)
    elif prim is False and date['is_weekday'] is True:
        return date['avail_people'].sort(key=get_duty_count_by_s_wd)
    else: 
        return date['avail_people'].sort(key=get_duty_count_by_s_we)

In [9]:
#testing sort_people in day

for person in schedule[2]['avail_people']:
    print(person['name'], person['s_we'])
sort_people_in_day(schedule[2], False)
for person in schedule[2]['avail_people']:
    print(person['name'] , person['s_we'])

Ryan 0
Annabel 0
Bridget 0
Paul 0
Chloe 0
Ryan 0
Annabel 0
Chloe 0
Paul 0
Bridget 0


In [10]:
def assign_shift(date, prim): 
    i_person = 0
    if prim is True:
        while date['sec'] == date['avail_people'][i_person]:
            if i_person is len(date['avail_people']):
                raise IndexError("Could not find person available that was not self")
            else:
                i_person+=1
        date['prim'] = date['avail_people'][i_person]
        shift_event = Event()
        shift_event['uid'] = uuid.uuid1()
        shift_event['summary'] = date['prim']['name'] + " - Primary"
        attendee = vCalAddress('MAILTO:'+date['prim']['email'])
        attendee.params['cn'] = vText(date['prim']['name'])
        attendee.params['ROLE'] = vText('REQ-PARTICIPANT')
        shift_event.add('attendee', attendee, encode=0)
        shift_event.add('dtstamp', dt.now())
        shift_event.add('dtstart', dt(date['date'].year, date['date'].month, date['date'].day, 19, 0,0))
        shift_event.add('dtend', dt(date['date'].year, date['date'].month, date['date'].day, 23, 59,59))
        shift_event['dtstamp'].to_ical()
        shift_event['dtstart'].to_ical()
        shift_event['dtend'].to_ical()
        sched_cal.add_component(shift_event)
        if date['is_weekday'] is True:
            date['avail_people'][i_person]['p_wd'] = date['avail_people'][i_person]['p_wd'] + 1
        else: 
            date['avail_people'][i_person]['p_we'] = date['avail_people'][i_person]['p_we'] + 1
        # eventually add to calendar
    else: 
        while date['prim'] == date['avail_people'][i_person]:
            if i_person is len(date['avail_people']):
                raise IndexError("Could not find person available that was not self")
            else:
                i_person+=1
        date['sec'] = date['avail_people'][i_person]
        shift_event = Event()
        shift_event['uid'] = uuid.uuid1()
        shift_event['summary'] = date['sec']['name'] + " - Secondary"
        attendee = vCalAddress('MAILTO:'+date['sec']['email'])
        attendee.params['cn'] = vText(date['sec']['name'])
        attendee.params['ROLE'] = vText('REQ-PARTICIPANT')
        shift_event.add('attendee', attendee, encode=0)
        shift_event.add('dtstamp', dt.now())
        shift_event.add('dtstart', dt(date['date'].year, date['date'].month, date['date'].day, 7, 0,0))
        shift_event.add('dtend', dt(date['date'].year, date['date'].month, date['date'].day, 23, 59,59))
        shift_event['dtstamp'].to_ical()
        shift_event['dtstart'].to_ical()
        shift_event['dtend'].to_ical()
        sched_cal.add_component(shift_event)
        if date['is_weekday'] is True:
            date['avail_people'][i_person]['s_wd'] = date['avail_people'][i_person]['s_wd'] + 1
        else: 
            date['avail_people'][i_person]['s_we'] = date['avail_people'][i_person]['s_we'] + 1
        # eventually add to calendar

In [11]:
#testing assign_shift

t_day = {'date': d(2023, 11, 1),
  'avail_people': [{'name': 'Ryan',
    'email':'test@test.io',
    'p_wd': 2,
    's_wd': 2,
    'p_we': 3,
    's_we': 4,
    'unavail': [d(2023, 11, 2),
     d(2023, 11, 5),
     d(2023, 11, 22),
     d(2023, 11, 15),
     d(2023, 11, 8)]},
   {'name': 'Annabel',
    'email':'test@test.io',
    'p_wd': 4,
    's_wd': 0,
    'p_we': 0,
    's_we': 0,
    'unavail': [d(2023, 11, 5),
     d(2023, 11, 10),
     d(2023, 11, 19),
     d(2023, 11, 11)]},
   {'name': 'Paul',
    'email':'test@test.io',
    'p_wd': 2,
    's_wd': 0,
    'p_we': 0,
    's_we': 0,
    'unavail': [d(2023, 11, 27), d(2023, 11, 28)]},
   {'name': 'Chloe',
    'email':'test@test.io',
    'p_wd': 9,
    's_wd': 0,
    'p_we': 0,
    's_we': 0,
    'unavail': [d(2023, 11, 9),
     d(2023, 11, 10),
     d(2023, 11, 21),
     d(2023, 11, 22),
     d(2023, 11, 23),
     d(2023, 11, 24),
     d(2023, 11, 25)]}],
  'prim': None,
  'sec': None,
  'is_weekday': True}

sort_people_in_day(t_day, True)
assign_shift(t_day, True)
t_day['prim'], t_day['sec']
sort_people_in_day(t_day, False)
assign_shift(t_day, False)
t_day['prim'], t_day['sec']

({'name': 'Ryan',
  'email': 'test@test.io',
  'p_wd': 3,
  's_wd': 2,
  'p_we': 3,
  's_we': 4,
  'unavail': [datetime.date(2023, 11, 2),
   datetime.date(2023, 11, 5),
   datetime.date(2023, 11, 22),
   datetime.date(2023, 11, 15),
   datetime.date(2023, 11, 8)]},
 {'name': 'Annabel',
  'email': 'test@test.io',
  'p_wd': 4,
  's_wd': 1,
  'p_we': 0,
  's_we': 0,
  'unavail': [datetime.date(2023, 11, 5),
   datetime.date(2023, 11, 10),
   datetime.date(2023, 11, 19),
   datetime.date(2023, 11, 11)]})

In [12]:
for day in schedule: 
    sort_people_in_day(day, True)
    assign_shift(day, True)
    sort_people_in_day(day, False)
    assign_shift(day, False)
    

In [13]:
for day in schedule:
    print(day['date'].day, day['prim']['name'], day['sec']['name'])


1 Chloe Ryan
2 Chloe Bridget
3 Paul Annabel
4 Annabel Paul
5 Bridget Paul
6 Paul Bridget
7 Ryan Chloe
8 Annabel Paul
9 Ryan Annabel
10 Bridget Ryan
11 Bridget Chloe
12 Bridget Annabel
13 Annabel Chloe
14 Chloe Bridget
15 Paul Annabel
16 Ryan Paul
17 Annabel Ryan
18 Chloe Bridget
19 Ryan Bridget
20 Bridget Ryan
21 Ryan Paul
22 Paul Annabel
23 Paul Annabel
24 Bridget Paul
25 Paul Ryan
26 Annabel Chloe
27 Chloe Ryan
28 Chloe Annabel
29 Paul Bridget
30 Chloe Bridget


In [14]:
with open("example-cal.ics", 'w') as cal_file:
    cal_file.write(sched_cal.to_ical().decode("utf-8"))